### 思维链（CoT）的例子

### 随机三位数的乘法

In [ ]:
import os
os.environ["LANGCHAIN_PROJECT"] = "macktalk-ai-course"
print(os.environ.get("LANGCHAIN_PROJECT"))
print(os.environ.get("SERPAPI_API_KEY"))

In [ ]:
import random

import openai, os
openai.api_key = os.environ.get("OPENAI_API_KEY")

COMPLETION_MODEL = "text-davinci-003"
def get_response(prompt, model = COMPLETION_MODEL, temperature = 0.0, stop=None):
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=stop,
        temperature=temperature,
    )
    message = completions['choices'][0]['text']  # type: ignore
    return message

# 生成两个三位数
total = 100
correct = 0

def extract_numbers(text):
    # 提取字符串中的所有数字
    numbers = ''.join(char for char in text if char.isdigit())
    # 将提取的数字转换为一个整数
    return int(numbers) if numbers else None

for i in range(total):
    num1 = random.randint(100, 999)
    num2 = random.randint(100, 999)

    # 进行乘法运算
    result = num1 * num2

    question = f"请计算{num1}乘以{num2}等于多少？答案是："""
    answer = get_response(question)
    chatgpt_result = extract_numbers(answer)
    if int(chatgpt_result) == int(result):
        correct += 1
    print(f"Answer: {answer}")
    print(f"ChatGPT 结果 {chatgpt_result}")
    print(f"计算器结果 {result}")
    # 输出结果
    #print(f"随机数1: {num1}, 随机数2: {num2}, 乘法结果: {result}")

print(f"正确率: {correct / total}")

### 思维链的例子

In [ ]:
import openai, os
openai.api_key = os.environ.get("OPENAI_API_KEY")

COMPLETION_MODEL = "text-davinci-002"
def get_response(prompt, model = COMPLETION_MODEL, temperature = 1.0, stop=None):
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=stop,
        temperature=temperature,
    )
    message = completions['choices'][0]['text']  # type: ignore
    return message

question = """A: 一个杂耍者有16个球。一半的球是高尔夫球，而高尔夫球的一半是蓝色的。那么有多少个蓝色高尔夫球呢？
Q:"""

print(get_response(question))

In [ ]:

question = """A: 一个杂耍者有16个球。一半的球是高尔夫球，而高尔夫球的一半是蓝色的。那么有多少个蓝色高尔夫球呢？
Q: 让我们一步一步想，"""

print(get_response(question))

In [ ]:
question = """利用以下线索回答下面的选择题。

线索：
1. Miss Scarlett是休息室中唯一的人。
2. 抽烟斗的人在厨房里。
3. Colonel Mustard是天文台中唯一的人。
4. Professor Plum不在图书馆和台球室。
5. 拿着蜡烛台的人在天文台。

问题：Colonel Mustard是否在天文台中持着蜡烛台？
(a) 是的；Colonel Mustard在天文台中持着蜡烛台。
(b) 不是的；Colonel Mustard没有在天文台中持着蜡烛台。
(c) 未知；没有足够的信息来确定Colonel Mustard是否在天文台中持着蜡烛台。

解答："""

print(get_response(question))

In [ ]:
question = """利用以下线索回答下面的选择题，使用以下步骤：
(1) 首先，逐一查看线索并考虑该线索是否有潜在关联性。
(2) 其次，结合相关线索来推理出问题的答案。
(3) 最后，将答案映射到多项选择答案中的一个：要么是(a)、(b)或(c)。

线索：
1. Miss Scarlett是休息室中唯一的人。
2. 抽烟斗的人在厨房里。
3. Colonel Mustard是天文台中唯一的人。
4. Professor Plum不在图书馆和台球室。
5. 拿着蜡烛台的人在天文台。

问题：Colonel Mustard是否在天文台中持着蜡烛台？
(a) 是的；Colonel Mustard在天文台中持着蜡烛台。
(b) 不是的；Colonel Mustard没有在天文台中持着蜡烛台。
(c) 未知；没有足够的信息来确定Colonel Mustard是否在天文台中持着蜡烛台。

解答：
(1) 首先，逐一查看线索并考虑该线索是否有潜在关联性："""

print(get_response(question))

### Least-to-most prompting

In [ ]:
## 注意，并不是每一次都会出错
question = """Q: It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. 
The water slide closes in 18 minutes. How many times can she slide before it closes? """
## Amy需要4分钟爬到滑梯的顶端，她需要1分钟滑下来。
## 水滑梯将在15分钟后关闭。在水滑梯关闭之前，她可以滑多少次？
print(get_response(question, model="text-davinci-003"))

In [ ]:
question = """It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. 
The water slide closes in 18 minutes. """

step1 = """\nA: To solve "How many times can she slide before it closes?", we need to first solve: \""""

question1 = "Q: " + question + step1

print(question1)
answer1 = get_response(question1, model="text-davinci-003", stop="\n")
print("\n\n")
print(answer1)

In [ ]:
question2 = question + "\nQ: " + answer1 + "?\nA: "
print(question2)

answer2 = get_response(question2, model="text-davinci-003", stop="\n")
print(answer2)


In [ ]:
question3 = question2 + answer2 + "\n Q: " + "How many times can she slide before it closes?" + "\nA: "
print(question3)

answer3 = get_response(question3, model="text-davinci-003", stop="\n")
print(answer3)

### 用Least-to-most 思路来写单元测试

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


def write_unit_test(function_to_test, unit_test_package = "pytest"):
    # 解释源代码的步骤
    explain_code = """"# How to write great unit tests with {unit_test_package}

    In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
    ```python
    {function_to_test}
    ```

    Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
    - First,"""

    explain_code_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test"],
        template=explain_code
    )
    explain_code_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000,  # type: ignore
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])  # type: ignore
    explain_code_step = LLMChain(llm=explain_code_llm, prompt=explain_code_template, output_key="code_explaination", verbose=True)


    # 创建测试计划示例的步骤
    test_plan = """
        
    A good unit test suite should aim to:
    - Test the function's behavior for a wide range of possible inputs
    - Test edge cases that the author may not have foreseen
    - Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
    - Be easy to read and understand, with clean code and descriptive names
    - Be deterministic, so that the tests always pass or fail in the same way

    `{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

    For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
    -"""
    test_plan_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination"],
        template= explain_code + "{code_explaination}" + test_plan
    )
    test_plan_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000,   # type: ignore
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])  # type: ignore
    test_plan_step = LLMChain(llm=test_plan_llm, prompt=test_plan_template, output_key="test_plan",verbose=True)

    # 撰写测试代码的步骤
    starter_comment = "Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator"
    prompt_to_generate_the_unit_test = """

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

#{starter_comment}"""

    unit_test_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination", "test_plan", "starter_comment"],
        template= explain_code + "{code_explaination}" + test_plan + "{test_plan}" + prompt_to_generate_the_unit_test
    )
    unit_test_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, stop="```")  # type: ignore
    unit_test_step = LLMChain(llm=unit_test_llm, prompt=unit_test_template, output_key="unit_test",verbose=True)

    sequential_chain = SequentialChain(chains=[explain_code_step, test_plan_step, unit_test_step],  # type: ignore
                                    input_variables=["unit_test_package", "function_to_test", "starter_comment"], verbose=True)
    answer = sequential_chain.run(unit_test_package=unit_test_package, function_to_test=function_to_test, starter_comment=starter_comment)
    return f"""#{starter_comment}""" + answer


code = """
def format_time(seconds):
    print(f"Formatting {seconds} seconds")
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
"""

import ast

def write_unit_test_automatically(code, retry=3):
    unit_test_code = write_unit_test(code)
    print(f"Generated unit test code:\n{unit_test_code}")
    all_code = code + unit_test_code
    tried = 0
    while tried < retry:
        try:
            ast.parse(all_code)
            return all_code
        except SyntaxError as e:
            print(f"Syntax error in generated code: {e}")
            all_code = code + write_unit_test(code)
            tried += 1

import pytest
exec(write_unit_test_automatically(code))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"# How to write great unit tests with pytest

    In this advanced tutorial for experts, we'll use Python 3.10 and `pytest` to write a suite of unit tests to verify the behavior of the following function.
    ```python
    
def format_time(seconds):
    print(f"Formatting {seconds} seconds")
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"

    ```

    Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
    - First,

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
"# How to write great unit tests with pytest

    In this advanced tutorial for experts, we'll use Pyth

### Self-consistency Prompt

In [ ]:
question = """Q: 如果停车场里有3辆汽车，又有2辆汽车到达，停车场里有多少辆汽车？
A: 停车场里已经有3辆汽车。又有2辆汽车到达。现在总共有3 + 2 = 5辆汽车。答案是5辆。

Q: Janet的鸭子每天下16个蛋。她每天早上吃掉其中三个，并用四个蛋每天烤松饼给她的朋友。她以每个蛋2美元的价格卖掉剩下的蛋。她每天能赚多少钱？
A: """


COMPLETION_MODEL = "text-davinci-002"
def get_top_response(prompt, n=10, model = COMPLETION_MODEL, temperature = 1.0, stop=None):
    messages = []
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=n,
        stop=stop,
        temperature=temperature,
    )
    for choice in completions['choices']: # type: ignore
        messages.append(choice['text'])
    return messages

answers = []

answers = get_top_response(question, n=10, model="text-davinci-003")
for answer, i in zip(answers, range(10)):
    print("Answer: " + str(i))
    print(answer)

vote_question = """
Janet的鸭子每天下16个蛋。她每天早上吃掉其中三个，并用四个蛋每天烤松饼给她的朋友。她以每个蛋2美元的价格卖掉剩下的蛋。她每天能赚多少钱？

请从以下5个人的答案中，选出票数最多答案作为最终答案。
"""

for answer, i in zip(answers, range(10)):
    vote_question += f"\n答案{i}: {answer}"

final_answer = get_response(vote_question, model="text-davinci-003")
print("Final answer:")
print(final_answer)


### Think step by step 计算三位数乘法

In [ ]:
import random

import openai, os
openai.api_key = os.environ.get("OPENAI_API_KEY")

COMPLETION_MODEL = "text-davinci-003"
def get_response(prompt, model = COMPLETION_MODEL, temperature = 0.0, stop=None):
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=stop,
        temperature=temperature,
    )
    message = completions['choices'][0]['text']  # type: ignore
    return message

# 生成两个三位数
total = 10
correct = 0

for i in range(total):
    num1 = random.randint(100, 999)
    num2 = random.randint(100, 999)

    # 进行乘法运算
    result = num1 * num2

    question = f"请计算{num1}乘以{num2}等于多少？请一步一步想。"""
    answer = get_response(question)
    print(f"Answer: {answer}")
    print(f"计算器结果 {result}")


### Temperature参数的影响

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def softmax_with_temp(x, temp):
    x = x / temp
    e_x = np.exp(x - np.max(x))  # subtract max to avoid overflow
    return e_x / e_x.sum(axis=0)

values = np.array([1.0, 2.0, 3.0])
words = ['apple', 'banana', 'cherry']
temps = [0.1, 1, 10]

# setting up the figure with 3 subplots (in a vertical stack)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 6))

# setting up the positions for groups and bars within groups
group_positions = np.arange(len(values))

# creating the bars
for ax, temp in zip(axes, temps):
    probs = softmax_with_temp(values, temp)
    ax.bar(group_positions, probs, label=f'Temp={temp}')
    ax.set_xlabel('Words')
    ax.set_ylabel('Probabilities')
    ax.set_title(f'Softmax probabilities for Temp={temp}')
    ax.set_xticks(group_positions)
    ax.set_xticklabels(words)
    ax.set_ylim([0, 1])  # setting y-axis limits
    ax.legend()

# adjusting the layout and showing the plot
plt.tight_layout()
plt.show()

### MRKL 例子

In [2]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType


In [3]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

In [4]:
mrkl = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [5]:
mrkl.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Who is Leo DiCaprio's girlfriend?"
Observation: Who is Leonardo DiCaprio currently dating? Leonardo DiCaprio has most recently been linked to 25-year-old Italian model Vittoria Ceretti. The couple were first spotted together in France in late May in the leadup to DiCaprio's appearance at the Cannes Film Festival.
Thought: I need to calculate Vittoria Ceretti's age raised to the 0.43 power.
Action: Calculator
Action Input: 25^0.43

> Entering new LLMMathChain chain...
25^0.43```text
25**0.43
```
...numexpr.evaluate("25**0.43")...

Answer: 3.991298452658078
> Finished chain.

Observation: Answer: 3.991298452658078
Thought: I now know the final answer.
Final Answer: Vittoria Ceretti's age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


"Vittoria Ceretti's age raised to the 0.43 power is 3.991298452658078."

### MRKL面对开放复杂的问题容易失败

In [6]:
mrkl.run(["Write a 5000 words report about the current progress of auto pilot for Tesla."])



> Entering new AgentExecutor chain...
 I need to research the current progress of auto pilot for Tesla
Action: Search
Action Input: 'Current progress of auto pilot for Tesla'
Observation: Tesla's Autopilot is classified as Level 2 under the SAE six levels (0 to 5) of vehicle automation. At this level, the car can act autonomously, but requires the driver to monitor the driving at all times and be prepared to take control at a moment's notice.
Thought: I need to find more information about the current progress of auto pilot for Tesla
Action: Search
Action Input: 'Recent developments in auto pilot for Tesla'
Observation: DETROIT (AP) — Tesla is allowing some drivers to use its Autopilot driver-assist system for extended periods without making them put their hands on the steering wheel, a development that has drawn concern from U.S. safety regulators.
Thought: I now know the final answer
Final Answer: Tesla's Autopilot is classified as Level 2 under the SAE six levels (0 to 5) of vehicl

"Tesla's Autopilot is classified as Level 2 under the SAE six levels (0 to 5) of vehicle automation. At this level, the car can act autonomously, but requires the driver to monitor the driving at all times and be prepared to take control at a moment's notice. Recent developments include Tesla allowing some drivers to use its Autopilot driver-assist system for extended periods without making them put their hands on the steering wheel, a development that has drawn concern from U.S. safety regulators."

In [7]:
mrkl.run(["写一篇5000字的关于当前特斯拉自动驾驶技术的最新进展的技术报告。"])



> Entering new AgentExecutor chain...
 I need to research the latest developments in Tesla's Autopilot technology
Action: Search
Action Input: 'Latest developments in Tesla Autopilot technology'
Observation: ['Autopilot is an advanced driver assistance system that enhances safety and convenience behind the wheel. When used properly, Autopilot reduces your overall ...']
Thought: I need to find more information about the technology
Action: Search
Action Input: 'How does Tesla Autopilot work?'
Observation: This advanced technology relies on eight cameras and robust neural net processing to perceive the car's surroundings and provide Autopilot features. By utilizing this camera suite, occupants are granted a heightened awareness of their environment, surpassing what a lone driver could perceive.
Thought: I now know enough to write the report
Final Answer: A 5000-word technical report on the latest developments in Tesla Autopilot technology should include information on how the technology

'A 5000-word technical report on the latest developments in Tesla Autopilot technology should include information on how the technology works, its features, and how it enhances safety and convenience for drivers.'

### AutoGPT 写报告

In [8]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [9]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [10]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [12]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI

In [13]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model_name="gpt-4", temperature=0),
    memory=vectorstore.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

In [14]:
agent.run(["Write a 5000 words report about the current progress of auto pilot for Tesla."])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Write a 5000 words report about the current progress of auto pilot for Tesla.


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_fil

InvalidRequestError: The model `gpt-4` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.

### Generative Agent

In [15]:
import logging
logging.basicConfig(level=logging.ERROR)

from datetime import datetime, timedelta
from typing import List
from termcolor import colored


from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS


USER_NAME = "Person A" # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(max_tokens=1500) # Can be any LLM you want.

In [16]:
from langchain_experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory


import math
import faiss

def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)


tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8 # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(name="Tommie", 
              age=25,
              traits="焦虑、喜欢设计、健谈", # You can add more persistent traits here 
              status="正在找工作", # When connected to a virtual world, we can have the characters update their status
              memory_retriever=create_new_memory_retriever(),
              llm=LLM,
              memory=tommies_memory
             )

In [17]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: 焦虑、喜欢设计、健谈
Tommie is straightforward, determined, and confident.


In [18]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie还记得小时候的狗Bruno。",
    "Tommie因为长途驾车感到疲倦。",
    "Tommie看到了新的住所。",
    "新的邻居有一只猫。",
    "夜晚道路很吵。",
    "Tommie感到饥饿。",
    "Tommie试图休息一下。",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [19]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: 焦虑、喜欢设计、健谈
Tommie is tired and in need of rest. They have memories of a dog named Bruno from their childhood. Tommie is also feeling hungry and has come across a new place to live. They notice that their new neighbor has a cat. Additionally, Tommie is bothered by the noisy road at night.


In [20]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [21]:
interview_agent(tommie, "你喜欢干什么?")


'Tommie said "我喜欢设计和艺术。你呢？"'

In [22]:
interview_agent(tommie, "你今天打算干些什么?")


'Tommie said "我今天打算休息一下。昨晚的夜路太吵了，我感到有些疲倦。你呢，今天有什么计划吗？"'

In [23]:
interview_agent(tommie, "你今天最担心的事情是什么?")


'Tommie said "我今天最担心的事情是找到一份工作。我正在积极寻找工作机会，但目前还没有成功。希望能尽快找到一份满意的工作。你呢？你今天最担心的是什么？"'